In [1]:
import torch

In [2]:
data = torch.load("data/act-mooc/graph.pt")

In [3]:
data

HeteroData(
  user={ node_id=[7047] },
  resource={ node_id=[97] },
  (user, accesses, resource)={
    edge_index=[2, 411749],
    edge_attr=[411749, 4],
    time=[411749],
    edge_y=[411749],
  },
  (resource, rev_accesses, user)={
    edge_index=[2, 411749],
    edge_attr=[411749, 4],
    time=[411749],
    edge_y=[411749],
  }
)

In [ ]:
# TODO:
# 1. define forward pass model
# 1a. message passing between all nodes by the edges they have excluding future edges of users and resources
# 1b. temporal link prediction based on embeddings
# 1c. temporal dropout prediction
# 2. training routine
# 2a. special sampling
# 3. test routine